# Gathering Tweet Data using twitter module tweety
Information about the Twitter api can be found at [docs.tweepy.org](http://docs.tweepy.org/en/v3.1.0/api.html#API.search)
Additional and updated information about Twitter's `Standard Search api` can be found [here](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets)


In [1]:
import tweepy
import pandas as pd

import matplotlib.pyplot as plt

# Get your Twitter API credentials and enter them here
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# authenticate the api with token and access information
api = tweepy.API(auth)

In [2]:
# API.home_timeline returns the 'n' most recent statuses, including retweets, 
# posted by the authenticating user and that user’s friends. 

public_tweets = api.home_timeline(count = 10)
for i, tweet in enumerate(public_tweets):
     print (i, tweet.text)

0 California HHS server crashed. Maybe it has covid. https://t.co/eGkdPmBpAK
1 Nothing beats well documented code... Thanks to @applepie and Pablo Rivera  for Tweepy Python Library and developer… https://t.co/WL4NGOjoF8
2 Actual data reference CHHS https://t.co/q7c7708ozx
3 Hospitals in California have been half empty this whole time https://t.co/8GL2BLypqV
4 RT @Lebeaucarnews: $TSLA CEO @elonmusk blasts shelter in place orders: “To say that they cannot leave their house and they will be arrested…
5 Our developer community is full of inspiring ideas that help the world learn from and improve our collective prepar… https://t.co/XObxbOQqmQ
6 This new streaming endpoint is available in Twitter Developer Labs and provides comprehensive access to COVID-19 re… https://t.co/wFylrCYJI8
7 The volume of conversation about COVID-19 is tremendous, which means it requires expertise and computational resour… https://t.co/Rzsa9P2Bn1
8 During these unprecedented times, what’s happening on Twitter can 

In [3]:
# Use the API.search eturns tweets that match a specified query
# reference: http://docs.tweepy.org/en/v3.1.0/api.html#API.search

covid_tweets = api.search('StayAtHome', lang='en')
for i, t in enumerate(covid_tweets):
    if i > 4: break
    print(i, t.text)

0 RT @QuickTake: 💕 "The meeting was at full capacity throughout the 2 hours and 30 minutes."

This couple in Mumbai held a #StayAtHome weddin…
1 RT @AlmaEgann: Tweeter Friends 
Happy Thursday 
Park in Japan 🇯🇵 

#StayAtHome https://t.co/FSTSsryJ0n
2 RT @FIFAWorldCup: 1990 ⏭ 2019

From Messi magic at the #U20WC to an epic @BelRedDevils comeback, we’ve got #WorldCup history spanning 29 ye…
3 RT @SeanOnTV: Do you agree with @dougducey extending the #StayAtHome order + travel restrictions until May 15th? #azfamily #COVID19
4 RT @UNESCO: The story of Jazz is written into the quest for human dignity, democracy &amp; civil rights.

Photographer Jürgen Schadeberg’s work…


In [4]:
# Convert twitter results to pandas dataframe for further analysis
results = api.search(q='StayAtHome', count=5)
json_data = [r._json for r in results]
df = pd.io.json.json_normalize(json_data)
print(df.loc[0, :])

contributors                                                                            None
coordinates                                                                             None
created_at                                                    Thu Apr 30 03:24:29 +0000 2020
entities.hashtags                          [{'text': 'StayAtHome', 'indices': [57, 68]}, ...
entities.media                                                                           NaN
entities.symbols                                                                          []
entities.urls                                                                             []
entities.user_mentions                     [{'screen_name': 'SeanOnTV', 'name': 'Sean McL...
extended_entities.media                                                                  NaN
favorite_count                                                                             0
favorited                                                             

## Create function to pull  twitter text given a query

In [8]:
def get_word_tweets(query='StayAtHome', num_result = 100):
    '''
    input: 
        api - twitter ai
        query - words to use for query 
        num_result 
    ouput: 
        a transformed data frame object
    '''
     
    json_data = []; n = 0;
    while n < num_result: 
        for search_result in api.search(query, lang='en', count=num_result-n):
            if n > num_result: 
                break 
            n = n + 1
            json_data.append(search_result._json)
    
    df = pd.io.json.json_normalize(json_data)
    
    return df[['id', 'created_at', 'text', 'favorite_count', 'retweet_count', 'truncated']]

In [9]:
df = get_word_tweets('Covid19', 10)
print(list(df.text.head()))

['RT @Omkar_Raii: .@Vivo_India donating 15,000 PPE suits and 50,000 litres of sanitizer to the central government to protect caregivers and h…', 'RT @drvishalrao: How Plasma Therapy Works\n\n#covid19 #plasmatherapy https://t.co/PcI4TVlCwP', 'RT @RahulGandhi: At 9 AM today, you can watch my conversation with Dr Raghuram Rajan, former RBI Governor, on dealing with the #Covid19 cri…', 'RT @BrentToderian: Yet another reminder that there’s a difference between density and crowding. #PhysicalDistancing #COVID19 #coronavirus V…', 'RT @RahulGandhi: At 9 AM today, you can watch my conversation with Dr Raghuram Rajan, former RBI Governor, on dealing with the #Covid19 cri…']
